In [1]:
import pandas as pd
import numpy as np
import re
from nltk.stem.porter import PorterStemmer # Kelimenin köküne erişebilmek için kullanılır. İngilizce’de en yaygın olanı Porter Stammer’dır.

In [2]:
data = pd.read_csv('gender_classifier.csv', encoding='latin1')
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


* İhtiyacımız olan gender ve description verileridir.

In [3]:
data1 = pd.concat([data.gender, data.description], axis=1)
data1.head(3)

,gender,description
0,male,i sing my own rhythm.
1,male,I'm the author of novels filled with family dr...
2,male,louis whining and squealing and all


In [4]:
data1.shape

(20050, 2)

In [5]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gender       19953 non-null  object
 1   description  16306 non-null  object
dtypes: object(2)
memory usage: 313.4+ KB


In [6]:
data1.isnull().sum()

gender           97
description    3744
dtype: int64

In [7]:
data1.dropna(inplace=True)

data1.isnull().sum()

gender         0
description    0
dtype: int64

In [8]:
# silme iişleminden sonra index değerleri bozuldu onlasrı düzeltelim.
data1.reset_index(drop=True, inplace=True)

In [9]:
data1.shape

(16224, 2)

* Gender verilerini sayısal verilere çevirelim

In [10]:
data1.gender= [1 if i == 'female' else 0 for i in data1.gender.values]

In [11]:
data1.head()

,gender,description
0,0,i sing my own rhythm.
1,0,I'm the author of novels filled with family dr...
2,0,louis whining and squealing and all
3,0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,1,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...


In [12]:
print("Before: \n", data1.description[3])
metin = re.sub("[^a-zA-Z]", " ", data1.description[3])
print("\nAfter: \n", metin)

Before: 
 Mobile guy.  49ers, Shazam, Google, Kleiner Perkins, Yahoo!, Sprint PCS, AirTouch, Air Force.  Stanford GSB, UVa.  Dad, Husband, Brother.  Golfer.

After: 
 Mobile guy     ers  Shazam  Google  Kleiner Perkins  Yahoo   Sprint PCS  AirTouch  Air Force   Stanford GSB  UVa   Dad  Husband  Brother   Golfer 


In [13]:
harfler = metin.lower()
print(harfler)

mobile guy     ers  shazam  google  kleiner perkins  yahoo   sprint pcs  airtouch  air force   stanford gsb  uva   dad  husband  brother   golfer 


In [14]:
bol = harfler.split()
print(bol)

['mobile', 'guy', 'ers', 'shazam', 'google', 'kleiner', 'perkins', 'yahoo', 'sprint', 'pcs', 'airtouch', 'air', 'force', 'stanford', 'gsb', 'uva', 'dad', 'husband', 'brother', 'golfer']


* Kelimelerin köklerini bulalım

In [15]:
ps = PorterStemmer() # Nesnesini oluşturuyoruz.

In [17]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sahik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords 
# Yüklenilen stopwords kütüphanelerini kullanabilmek için pythona dahil edelim.
# corpus metodu stopwordslerin bulunduğu bir paket anlamına geliyor.

In [20]:
print("Before: \n", bol)
metin = [ps.stem(i) for i in bol if not i in set(stopwords.words("english"))]
print("\nAfter: \n", metin)
metinson = " ".join(metin)
print("\nMetni Birleştirelim: \n", metinson)

Before: 
 ['mobile', 'guy', 'ers', 'shazam', 'google', 'kleiner', 'perkins', 'yahoo', 'sprint', 'pcs', 'airtouch', 'air', 'force', 'stanford', 'gsb', 'uva', 'dad', 'husband', 'brother', 'golfer']

After: 
 ['mobil', 'guy', 'er', 'shazam', 'googl', 'kleiner', 'perkin', 'yahoo', 'sprint', 'pc', 'airtouch', 'air', 'forc', 'stanford', 'gsb', 'uva', 'dad', 'husband', 'brother', 'golfer']

Metni Birleştirelim: 
 mobil guy er shazam googl kleiner perkin yahoo sprint pc airtouch air forc stanford gsb uva dad husband brother golfer


In [23]:
liste = []
for j in range(1000):
    metin = re.sub("[^a-zA-Z]", " ", data1.description[j])
    metin = metin.lower()
    metin = metin.split()
    metin = [ps.stem(i) for i in metin if not i in set(stopwords.words("english"))]
    metinson = " ".join(metin)
    liste.append(metinson)

print(liste)

['sing rhythm', 'author novel fill famili drama romanc', 'loui whine squeal', 'mobil guy er shazam googl kleiner perkin yahoo sprint pc airtouch air forc stanford gsb uva dad husband brother golfer', 'ricki wilson best frontman kaiser chief best band xxxx thank kaiser chief incred year gig memori cherish alway xxxxxxx', 'know', 'global marketplac imag video music share photo inspir design tip video creativ commun', 'secret get ahead get start', 'pll fan crazi mcd ramen bae', 'renaiss art historian univers nottingham fuell haribo partial coffe soft spot renaiss china nation teach fellow', 'clean food tast great provid energi nutrient guilt granola vegan paleo friendli option cert organ gf kosher', 'highli extraordinari auction', 'senior xi xii mmxiv', 'come join fastest blog network onlin today http co mfpa vgk http co mpuuqtyf g cover credit repair credit card bankruptci', 'im p bo burnham disney world', 'jmkm', 'enthusiast f fan model collector music fan film fanat also aspergian', 'a

In [29]:
# Veri setinde en sık kullanılan kelimeler

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer() # nesne oluşturalım.
# bağımsız değişken : konuşmalar
x = cv.fit_transform(liste).toarray() # listedeki verileri al ve array e çevir.
# bağımlı değişken : cinsiyet
y = data1.iloc[:1000, 0].values # 0. sütuna göre bin satır al

In [30]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [31]:
y

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,

In [32]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=33)

In [37]:
# Navibayes - istatistiksel hesaplamarı için kullanılır 
# Bununla elimizdeki verilerin doğruluğunu önce fit edeceiz sonra da hesaplayacağız. 

from sklearn.naive_bayes import GaussianNB
gn = GaussianNB()

gn.fit(x_train, y_train)
yhead= gn.predict(x_test)

In [38]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, yhead)
print(cm)

[[89 42]
 [23 46]]


-> kadınların söylediğinden 89 doğru 42 hatalı bulmuş.

-> erkeklerde 46 doğru 23 hatalı bulmuş.